In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
URL = 'https://gg.bet/en/counter-strike'
soup = BeautifulSoup(requests.get(URL).content, 'html.parser')

In [14]:
# soup

In [7]:
# soup

In [8]:
# for data in soup.html.body:
#     print(data)
#     print(20*'-')
# soup

In [9]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') #executable_path=yourdriver
driver.get(URL)
html = driver.page_source

In [10]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [13]:
table = driver.find_element_by_id('betting__container').text

In [14]:
soup = BeautifulSoup(table, 'html.parser')

In [15]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def remove_header(html, cutoff='RESULTS', separator='\n', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    for idx, token in enumerate(tokenized_html):
        if token == cutoff:
            break
    headless_html = padding_token.join(tokenized_html[idx+1:])
    return headless_html
    
def insert_row_breaks(html, separator='_PADDING_', row_break_token='_ROW_BREAK_', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    num_tokens = len(tokenized_html)
    insert = False
    processed_html = [tokenized_html[0]]
    for idx in range(1, num_tokens):        
        if insert and not is_number(tokenized_html[idx]):
            insert = False
            processed_html.append(row_break_token)
        elif tokenized_html[idx - 1] == 'Over' and tokenized_html[idx] == 'Under':
            insert = True
            continue
        processed_html.append(tokenized_html[idx])
    processed_html = padding_token.join(processed_html)
    return processed_html

def idx_search(l, s, warn_on_failure=True):
    """Get the index of the first occurance of a string of characters in a list."""
    
    res_idx = -1
    for idx, token in enumerate(l):
        if s in token:
            res_idx = idx
            break
    if warn_on_failure and res_idx == -1:
        raise IndexSearchFailure()
            
    return res_idx

def get_tournament_name(token_list, stopwords, default="NA"):
    """Extract the tournament name."""
    
    tournament_name = default
    filtered_tokens = set(token_list) - stopwords
    if len(filtered_tokens) == 1:  # exact match
        tournament_name = filtered_tokens.pop()
    elif len(filtered_tokens) > 1:  # not exact match - additional cleaning
        tournament_name = default  # TODO: implement additional cleaning
        
    return tournament_name

def get_time():
    return
    
    
class IndexSearchFailure(Exception):
    pass

class FormattingError(Exception):
    pass


def encode_row(row, separator='_PADDING_', stopwords={'_', 'LIVE', '', '-'}):
    # TODO: NA is previous tournament, means no column headers
    
    result = {}
    row = row.split(separator)
    
    time_idx = idx_search(row, ':')  # time index search might fail if the game is live
    result['tournament_name'] = get_tournament_name(row[:time_idx], stopwords)
    result['time'] = row[time_idx]
    result['date'] = row[time_idx + 1]
    
    row = row[time_idx + 2:]
    winner_idx = idx_search(row, 'WINNER')
    if 'Total' in row[:winner_idx]:  # bad fit -> correct
        contestants = []
        for winner_idx, token in enumerate(row):
            if is_number(token):
                winner_idx += 1
                break
            contestants.append(token)
        result['contestants'] = contestants
    else:  
        result['contestants'] = row[:winner_idx]

    row = row[winner_idx+1:]
 
    # winner odds
    result['winner_odds'] = row[:2]

    row = row[2:]

    # round handicap
    result['handicap_type'] = row[0]
    result['handicap_odds'] = row[1:5]
    
    row = row[5:]
    
    if len(row) < 6:  # fails on multi-team tournaments
        raise FormattingError()
    
    result['map_type'] = row[0]
    result['map_total'] = row[3]
    result['map_over_odds'] = row[4]
    result['map_under_odds'] = row[5]
    
    return result

In [71]:
text = remove_header(soup.text)
text = insert_row_breaks(text)
rows = text.split('_ROW_BREAK_')
table_data = []
for row in rows:
    try:
        result = encode_row(row)
        result['team_1'] = result['contestants'][0]
        result['team_1_winner_odds'] = result['winner_odds'][0]
        result['team_2'] = result['contestants'][2]
        result['team_2_winnder_odds'] = result['winner_odds'][1]
        if result['date'] == "TODAY":
            continue
        result['time'] = datetime.datetime.strptime(result['date'] + ' ' + result['time'], '%b %d %H:%M').replace(year=2019)
        result['time'] = datetime.datetime.timestamp(result['time'])
        del result['contestants']
        del result['winner_odds']
        del result['handicap_odds']
        del result['handicap_type']
        del result['map_type']
        del result['map_total']
        del result['map_over_odds']
        del result['map_under_odds']
        table_data.append(result)
    except (IndexSearchFailure, FormattingError):
        pass

In [72]:
df = pd.DataFrame(table_data)
numeric_cols = ['team_1_winner_odds', 'team_2_winnder_odds']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df

,date,team_1,team_1_winner_odds,team_2,team_2_winnder_odds,time,tournament_name
0,JUL 11,TRICKED,1.63,Nordavind,2.17,1.562842e+09,UCC SUMMER SMASH
1,JUL 11,Nemiga,1.20,AGO,4.20,1.562843e+09,GAMEAGENTS LEAGUE SEASON 3
2,JUL 11,Illuminar,2.60,Aristocracy,1.45,1.562853e+09,UCC SUMMER SMASH
3,JUL 11,Nordavind,2.76,forZe,1.41,1.562860e+09,LOOT.BET SMACK MY BEACH CUP
4,JUL 11,Lyngby Vikings,6.07,TRICKED,1.10,1.562868e+09,NA


In [73]:
df.dtypes

date                    object
team_1                  object
team_1_winner_odds     float64
team_2                  object
team_2_winnder_odds    float64
time                   float64
tournament_name         object
dtype: object

In [20]:
df.shape

(6, 11)

In [60]:
import datetime

dt = datetime.datetime.strptime('JUL 11 22:30', '%b %d %H:%M').replace(year=2019)
dt

datetime.datetime(2019, 7, 11, 22, 30)

In [ ]:

def db_insert(data):
	"""Insert ggbet data into database."""

	conn = None
	insert_statement = """
		INSERT INTO csgo_winner_odds (team_1, team_2, team_1_winner_odds, team_2_winner_odds, match_time, tournament_name, source)
		VALUES (%s, %s, %s, %s, %s, %s, %s);
	"""
    
	try:
		conn = psycopg2.connect(**DATABASE)
		cursor = conn.cursor()
		cursor.executemany(insert_statement, data)
		conn.commit()
		cursor.close()
		logger.info("Inserted %s rows.", len(data))
	except psycopg2.DatabaseError:
		logger.error("Failed db insert.")
	finally:
		if conn:
			conn.close()

In [14]:
soup.text

'FILTERS\nLIVE\nOF THE DAY\nWINNER\nOnLANers\n3.01\nElectronik Generation\n1.35\n+13\nUPCOMING GAMES\nOUTRIGHTS\nRESULTS\nLIGA PORTUGUESA SUMMER 2019 - DIVISION 2\nLIVE\nTODAY\nOnLANers\nX\nElectronik Generation\nSCORE\n0\n0\nWINNER\n3.01\n1.35\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n+3.5\n1.89\n-3.5\n1.84\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n27.5\n2.05\n1.71\nSTREAM\n11\nBRASIL GAME CUP 2019\nLIVE\n16:00\nTODAY\nIsurus\nX\nBulldozer\nWINNER\n1.1\n6.15\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n-7.5\n1.75\n+7.5\n1.99\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n23.5\n1.77\n1.96\n15\nAMD RED LEAGUE NORTHERN CONE 2019\nLIVE\n18:00\nTODAY\nAnunnaki\nX\nDivine Soldiers\nWINNER\n1.42\n2.73\nHANDICAP\n-\n-\nTOTAL\nTotal\nOver\nUnder\n-\n2\nESL AUSTRALIA & NZ CHAMPIONSHIP\nLIVE\n01:30\nJUL 09\nChiefs\nX\nGround Zero\nWINNER\n1.19\n4.31\nMAP HANDICAP\n-1.5\n1.67\n+1.5\n2.1\nTOTAL MAPS\nTotal\nOver\nUnder\n2.5\n2.42\n1.51\n38\nESL ONE NEW YORK 2019